# Model Visualization

In [ ]:
# IMPORT LIBRARIES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import importlib
import os
import random
import pickle
import copy

# IMPORT TORCH
import torch
from torch.utils.data import DataLoader

# IMPORT SELFEEG 
import selfeeg.dataloading as dl

# IMPORT REPOSITORY FUNCTIONS
import AllFnc
from   AllFnc import split
from   AllFnc.models import ShallowNet2
from   AllFnc.training import loadEEG

# IMPORT EEGVISLIB
from   AllFnc import eegvislib

import warnings
warnings.filterwarnings("ignore", message = "Using padding='same'", category = UserWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

In [ ]:
device = torch.device('cpu')

# SET PATH
sep         = os.path.sep
dataPath    = '/data/delpup/datasets/eegpickle/'
osPath      = os.path.abspath(os.getcwd())
imgPath     = osPath + sep + 'imgs' + sep
modelsPath  = osPath + sep + 'AlzClassification' + sep + 'Models' + sep
resultsPath = osPath + sep + 'AlzClassification' + sep + 'Results' + sep

# IMGS OPTIONS
imgs_format = '.pdf'
save_img    = True

## Set training spec

In [ ]:
pipelineToEval = 'filt' #'icasr'
taskToEval     = 'alzheimer'
outFold        = 6
inFold         = 5
downsample     = True
z_score        = True
rem_interp     = True
batchsize      = 64
overlap        = 0.0
workers        = 0
window         = 4
verbose        = True
arch_acronym   = 'shn7db'
seed           = 83136297

modelToimport = 'alz_flt_125_shn7db_006_005_000050_019_004'

# ShallowNet
# shallownet_custom_dict = {
#     "F1": 40,
#     "K1": 25,
#     "F2": 40,
#     "Pool": 75,
#     "p": 0.2,
#     "log_activation_base": "e",
#     "norm_type": "batchnorm",
#     "random_temporal_filter": True,
#     "Fs": 125 if downsample else 250 ,
#     "freeze_temporal": 0,
#     "dense_hidden": None,
#     "spatial_depthwise": False,
#     "spatial_only_positive": False,
#     "global_pooling": False,
#     "bias": [True, True, True],
#     "return_logits": True,
#     "seed": 83136297
# }

# Med-ShallowNet
shallownet_custom_dict = {
    "F1": 7,
    "K1": 125,
    "F2": 7,
    "Pool": 75,
    "p": 0.2,
    "log_activation_base": "dB",
    "norm_type": "batchnorm",
    "random_temporal_filter": False,
    "Fs": 125 if downsample else 250 ,
    "freeze_temporal": 999999999,
    "dense_hidden": None,
    "spatial_depthwise": True,
    "spatial_only_positive": False,
    "global_pooling": True,
    "bias": [False, False, False],
    "return_logits": True,
    "seed": seed
}

In [ ]:
# fold to eval is the correct index to get the desired train/val/test part
outerFold  = outFold - 1
innerFold  = inFold  - 1
foldToEval = outerFold*5 + innerFold

## Create partition list

In [ ]:
if 'alzheimer' in taskToEval.casefold():
    # ALZ = subjects 1 to 36; CTL = subjects 37 to 65; FTD = subjects 66 to 88
    a_id = [i for i in range(1,37)]
    c_id = [i for i in range(37,66)]
    f_id = [i for i in range(66,89)]
    
    part_a = split.create_nested_kfold_subject_split(a_id, 10, 5)
    part_c = split.create_nested_kfold_subject_split(c_id, 10, 5)
    part_f = split.create_nested_kfold_subject_split(f_id, 10, 5)

    if taskToEval.casefold() == 'alzheimer':
        partition_list_1 = split.merge_partition_lists(part_a, part_c, 10, 5)
        partition_list = split.merge_partition_lists(partition_list_1, part_f, 10, 5)
    elif taskToEval.casefold() == 'alzheimerca':
        partition_list = split.merge_partition_lists(part_a, part_c, 10, 5)
    elif taskToEval.casefold() == 'alzheimercf':
        partition_list = split.merge_partition_lists(part_c, part_f, 10, 5)
    elif taskToEval.casefold() == 'alzheimeraf':
        partition_list = split.merge_partition_lists(part_a, part_f, 10, 5)

elif taskToEval.casefold() == 'cognitive':
    # CTL = subjects 101 to 149; PD/PDD/PDMCI = mixing number in [1; 100]
    c_id = [i for i in range(101,150)]
    pd_id = [3, 6, 7, 16, 17, 18, 21, 24, 26, 27, 30, 32, 35, 37, 39, 40, 45,
             46, 50, 51, 53, 57, 58, 60, 61, 62, 63, 65, 67, 68, 69, 71, 74,
             76, 80, 81, 82, 83, 84, 85, 86, 87, 90, 92, 93, 94, 100]
    pdd_id = [1, 2, 5, 8, 9, 12, 13, 15, 22, 25, 33, 38, 44, 48, 75, 78, 88, 95, 96]
    pdmci_id = [4, 10, 11, 14, 19, 20, 23, 28, 29, 31, 34, 36, 41, 42, 43, 47,
                49, 52, 54, 55, 56, 59, 64, 66, 70, 72, 73, 77, 79, 89, 91, 97,
                98, 99]
    part_c = split.create_nested_kfold_subject_split(c_id, 10, 5)
    part_p = split.create_nested_kfold_subject_split(pd_id, 10, 5)
    part_d = split.create_nested_kfold_subject_split(pdd_id, 10, 5)
    part_m = split.create_nested_kfold_subject_split(pdmci_id, 10, 5)
    
    # first --> mix two groups; then --> mix the mix
    # splits have a more similar number of subject per set in this way
    partition_1 = split.merge_partition_lists(part_c, part_m, 10, 5)
    partition_2 = split.merge_partition_lists(part_p, part_d, 10, 5)
    partition_list = split.merge_partition_lists(partition_1, partition_2, 10, 5)

## Get other dataset info

In [ ]:
# Define the Path to EEG data as a concatenation of:
# 1) the root path
# 2) the preprocessing pipeline
if dataPath[-1] != os.sep:
    dataPath += os.sep
if pipelineToEval[-1] != os.sep:
    eegpath = dataPath + pipelineToEval + os.sep
else:
    eegpath = dataPath + pipelineToEval

# Define the number of Channels to use. 
# Basically 61 due to BIDSAlign channel system alignment.
# Note that BIDSAlign DOES NOT delete any original channel by default.
if rem_interp:
    if 'alzheimer' in taskToEval.casefold():
        Chan = 19
    elif taskToEval.casefold() == 'cognitive':
        Chan = 59
else:
    Chan = 61

# Define the sampling rate. 125 or 250 depending on the downsample option
srate = 125 if downsample else 250

# Define the number of classes to predict.
# All tasks are binary except the Alzheimer's one, 
# which is a multi-class classification (Alzheimer vs FrontoTemporal vs Control)
if taskToEval.casefold() == 'alzheimer':
    nb_classes = 3
elif taskToEval.casefold() == 'cognitive':
    nb_classes = 4
else:
    nb_classes = 2

# For selfEEG's models instantiation
Samples = int(srate*window)

# Set the Dataset ID for glob.glob operation in SelfEEG's GetEEGPartitionNumber().
if 'alzheimer' in taskToEval.casefold():
    datasetID = '10'
elif taskToEval.casefold() == 'cognitive':
    datasetID = '19'

# Set the class label in case of plot of functions
if taskToEval.casefold() == 'alzheimer':
    classlabels = ['CTL', 'FTD', 'AD']
elif taskToEval.casefold() == 'alzheimerca':
    classlabels = ['CTL', 'AD']
elif taskToEval.casefold() == 'alzheimercf':
    classlabels = ['CTL', 'FTD']
elif taskToEval.casefold() == 'alzheimeraf':
    classlabels = ['FTD', 'AD']
elif taskToEval.casefold() == 'cognitive':
    classlabels = ['CTL', 'PD', 'PDD', 'PDMCI']

## Define torch dataloaders

In [ ]:
loadEEG_args = {'return_label': False, 
                'downsample': downsample, 
                'use_only_original': rem_interp,
                'apply_zscore': z_score}

glob_input = [datasetID + '_*.pickle']

# calculate dataset length.
# Basically it automatically retrieves all the partitions 
# that can be extracted from each EEG signal
EEGlen = dl.get_eeg_partition_number(
    eegpath,
    srate,
    window,
    overlap, 
    file_format = glob_input,
    load_function = loadEEG,
    optional_load_fun_args = loadEEG_args,
    includePartial = False if overlap == 0 else True,
    verbose = verbose
)

# Now we also need to load the labels
loadEEG_args['return_label'] = True

# Set functions to retrieve dataset, subject, and session from each filename.
# They will be used by GetEEGSplitTable to perform a subject based split
dataset_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[0])
subject_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[1]) 
session_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[2]) 

# Now call the GetEEGSplitTable. 
if taskToEval.casefold() == 'alzheimerca':
    exclude_id = f_id
elif taskToEval.casefold() == 'alzheimercf':
    exclude_id = a_id
elif taskToEval.casefold() == 'alzheimeraf':
    exclude_id = c_id
else:
    exclude_id = None
        
    train_id   = partition_list[foldToEval][0]
    val_id     = partition_list[foldToEval][1]
    test_id    = partition_list[foldToEval][2]
    EEGsplit = dl.get_eeg_split_table(
        partition_table      = EEGlen,
        val_data_id          = val_id,
        test_data_id         = test_id,
        exclude_data_id      = exclude_id,
        split_tolerance      = 0.001,
        dataset_id_extractor = subject_id_ex,
        subject_id_extractor = session_id_ex,
        perseverance         = 10000)

if verbose:
    print(' ')
    print('Subjects used for test')
    print(test_id)

In [ ]:
# Define Datasets and preload all data
trainset = dl.EEGDataset(
    EEGlen, EEGsplit, [srate, window, overlap], 'train', 
    supervised             = True, 
    label_on_load          = True,
    load_function          = loadEEG,
    optional_load_fun_args = loadEEG_args
)
trainset.preload_dataset()

valset = dl.EEGDataset(
    EEGlen, EEGsplit, [srate, window, overlap], 'validation',
    supervised             = True, 
    label_on_load          = True,
    load_function          = loadEEG,
    optional_load_fun_args = loadEEG_args
)
valset.preload_dataset()

testset = dl.EEGDataset(
    EEGlen, EEGsplit, [srate, window, overlap], 'test',
    supervised             = True,
    label_on_load          = True,
    load_function          = loadEEG,
    optional_load_fun_args = loadEEG_args
)
testset.preload_dataset()

# Convert to long if task is multiclass classification.
# This avoids Value Errors during cross entropy loss calculation
if ('alzheimer' in taskToEval.casefold()) or ('cognitive' in taskToEval.casefold()):
    if taskToEval.casefold() == 'alzheimerca':
        trainset.y_preload[trainset.y_preload==2] = 1
        valset.y_preload[valset.y_preload==2] = 1 
        testset.y_preload[testset.y_preload==2] = 1
        
    elif taskToEval.casefold() == 'alzheimeraf':
        trainset.y_preload -= 1
        valset.y_preload   -= 1 
        testset.y_preload  -= 1
        
    elif taskToEval.casefold() == 'alzheimeraf':
        pass
        
    else:
        trainset.y_preload = trainset.y_preload.to(dtype = torch.long)
        valset.y_preload   = valset.y_preload.to(dtype = torch.long)
        testset.y_preload  = testset.y_preload.to(dtype = torch.long)
    
# Finally, Define Dataloaders
# (no need to use more workers in validation and test dataloaders)
trainloader = DataLoader(dataset = trainset, batch_size = batchsize,
                         shuffle = True, num_workers = workers)
valloader = DataLoader(dataset = valset, batch_size = batchsize,
                       shuffle = False, num_workers = 0)
testloader = DataLoader(dataset = testset, batch_size = batchsize,
                        shuffle = False, num_workers = 0)

if verbose:
    # plot split statistics
    labels = np.zeros(len(EEGlen))
    for i in range(len(EEGlen)):
        path = EEGlen.iloc[i,0]
        with open(path, 'rb') as eegfile:
            EEG = pickle.load(eegfile)
        labels[i] = EEG['label']
    dl.check_split(EEGlen, EEGsplit, labels)

## Import Model

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# define model
Mdl     = ShallowNet2(nb_classes, Chan, Samples, **shallownet_custom_dict)
Mdl_weights = torch.load(modelsPath + modelToimport + '.pt')
Mdl.load_state_dict(Mdl_weights)
Mdl_weights = {k: v for k, v in Mdl.state_dict().items()}
Mdl_weights = dict(Mdl_weights)

MdlBase = copy.deepcopy(Mdl)
MdlBase_weights = {k: v for k, v in MdlBase.state_dict().items()}
MdlBase_weights = dict(MdlBase_weights)

with open(resultsPath + modelToimport+'.pickle', 'rb') as file:
    scores = pickle.load(file)

inout_labels = [[classlabels[int(i)] for i in scores['labels']],
                [classlabels[int(i)] for i in scores['predictions']]]

## Visualization

In [ ]:
flim = [1,45]

bands   = {'$\\delta$' : [flim[0],4],
           '$\\theta$' : [4,8],
           '$\\alpha$' : [8,12],
           '$\\beta_1$': [12,16],
           '$\\beta_2$': [16,20],
           '$\\beta_3$': [20,28],
           '$\\gamma$' : [28,flim[1]]}

channels = ['Fp1','Fp2','F7','F3','Fz',
            'F4','F8','T7','C3','Cz',
            'C4','T8','P7','P3','Pz',
            'P4','P8','O1','O2']

montage = 'biosemi64'

spec_dict_keys  = ['bands','channels','montage','flim', 'srate', 'classlabels']
spec_dict_bands = [ bands,  channels,  montage,  flim,   srate,   classlabels]

In [ ]:
input_window_normalized = testloader.dataset[:][0]

### Input

In [ ]:
batch_IDs = [100, 1000]

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']   = [5,5]
spec_dict['font']     = 30
spec_dict['title']    = 'Input Signal'
spec_dict['cbartick'] = 5
spec_dict['f_des']    = [1,6,10,15,18,20,45]
spec_dict['contours'] = 0
spec_dict['lognorm']  = '10log10'
spec_dict['vlim']     = 'all' # 'all' or 'band' or 'batchID'
spec_dict['cmap']     = 'RdBu_r'
spec_dict['figname']  = 'scalpPSD'

for batch_ID in batch_IDs:
    fig = eegvislib.scalp_plot(input_window_normalized, 
                               spec_dict, inout_labels,
                               batch_ID)
    
    if save_img:
        fig.savefig(imgPath + spec_dict['figname'] + f'_batch{batch_ID}' + f'_split{outFold}_{inFold}' + imgs_format,
                    transparent=False, bbox_inches='tight')

### Conv1

In [ ]:
layer = 'encoder.conv1'

KERNELS

In [ ]:
filter_ID     = [0,1,2,3,4,5,6] #None (all Kernels), a number 0, or a list [0], [0,2]
kernel_ID     = 0
kernel_height = 0

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']      = [5,5]
spec_dict['linew']       = 3
spec_dict['font']        = 20
spec_dict['title']       = 'First Layer Filters ShallowNet'
spec_dict['rotation']    = 45
spec_dict['lognorm']     = '10log10'
spec_dict['bodeplot']    = 'mag' #both, phase, mag
spec_dict['colors']      = [['tab:red','black'],['tab:orange','tab:blue']]
spec_dict['linestyle']   = ['-','--']
spec_dict['loc']         = 'best'
spec_dict['filternames'] = [i for i in spec_dict['bands'].keys()] #or None
spec_dict['figname']     = 'firstLayerFiltersBodeplot'

fig, ax = eegvislib.BP_temporalkernels(Mdl_weights, layer,
                                       spec_dict, 
                                       filter_ID, kernel_ID, kernel_height, 
                                       None)

if save_img:
    fig.savefig(imgPath + spec_dict['figname']+ f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

In [ ]:
filter_ID     = None
kernel_ID     = 0
kernel_height = 0

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']  = [3,3]
spec_dict['font']    = 14
spec_dict['ytick']   = 5
spec_dict['s']       = 2
spec_dict['cmap']    = 'RdBu_r'
spec_dict['figname'] = 'firstLayerFiltersStemplot'

fig = eegvislib.weights_temporalkernels(Mdl_weights, layer, 
                                        spec_dict, 
                                        filter_ID, kernel_ID, kernel_height)

if save_img:
    fig.savefig(imgPath + spec_dict['figname']+ f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

ACTIVATIONS

In [ ]:
convoluted_output  = eegvislib.out_activation(Mdl, layer, input_window_normalized)
convoluted_output.shape
#batch x feature maps x scalp channels x time step

In [ ]:
# Parameters
chan_ID   = ['Fp1'] #or multiple channels ['Oz','Fpz']
batch_IDs = [100, 1000]
FM_ID     = [0,1,2,3,4,5,6] #None (all FM) or a number 0 or list [0], [2,4]

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']  = [4,3]
spec_dict['linew']   = 2
spec_dict['font']    = 14
spec_dict['lognorm'] = '10log10'
spec_dict['loc']     = 'upper right'
spec_dict['figname'] = 'firstLayerActivationsChannels'

for batch_ID in batch_IDs:
    fig = eegvislib.temporalFM_PSD(input_window_normalized, 
                                   Mdl, layer,   
                                   spec_dict, inout_labels,
                                   chan_ID, batch_ID, FM_ID)
    
    if save_img:
        fig.savefig(imgPath + spec_dict['figname'] + f'_batch{batch_ID}'+ f'_split{outFold}_{inFold}' + imgs_format,
                    transparent=False, bbox_inches='tight')

In [ ]:
batch_IDs = [100, 1000]
FM_ID     = [3] #None (all FM) or a number 0 or list [0], [2,4]

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']   = [5,5]
spec_dict['font']     = 30
spec_dict['cbartick'] = 5
spec_dict['contours'] = 0  
spec_dict['lognorm']  = '10log10'
spec_dict['vlim']     = 'band' # 'all' or 'band' or 'batchID'
spec_dict['cmap']     = 'RdBu_r'
spec_dict['figname']  = 'firstLayerActivationsPSD'

for batch_ID in batch_IDs:
    fig = eegvislib.temporalFM_PSDv2(input_window_normalized, 
                                     Mdl, layer, 
                                     spec_dict, inout_labels,
                                     batch_ID, FM_ID)
    if save_img:
        fig.savefig(imgPath + spec_dict['figname'] + f'_batch{batch_ID}'+ f'_split{outFold}_{inFold}' + imgs_format,
                    transparent=False, bbox_inches='tight')

### Conv2

In [ ]:
layer = 'encoder.conv2'

KERNEL

In [ ]:
filter_ID    = None #None (all Kernels), a number 0, or a list [0], [0,2]
kernel_ID    = [0] #None (all FM), a number 0, or a list [0], [0,2]
kernel_width = 0

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']      = [5,5]
spec_dict['font']        = 30
spec_dict['title']       = 'Second Layer Filters ShallowNet'
spec_dict['cbartick']    = 5
spec_dict['norm']        = 'nothing' #nothing, or softmax or abs
spec_dict['min_softmax'] = 1/len(channels) # 0 or 1/len(list_channels) (random chance)
spec_dict['filternames'] = [i for i in spec_dict['bands'].keys()] #or None
spec_dict['contours']    = 0
spec_dict['vlim']        = [-0.4,0.4]
spec_dict['cmap']        = ['RdBu_r','Reds'] #two colormaps, one for weights, and one for abs(weights)
spec_dict['figname']     = 'secondLayerFilters'

fig = eegvislib.spatialkernels_scalp(Mdl_weights, layer, 
                                     spec_dict,
                                     filter_ID, kernel_ID, kernel_width, 
                                     None)

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

ACTIVATION

In [ ]:
convoluted_output  = eegvislib.out_activation(Mdl, layer, input_window_normalized)
convoluted_output.shape
#batch x feature maps x scalp channels x time step

In [ ]:
batch_IDs = [100, 1000]
FM_ID = None #None (all FM) or a number 0 or list [0], [2,4]

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']   = [10,3]
spec_dict['font']     = 14
spec_dict['title']    = 'PSD_FM'
spec_dict['cbartick'] = 5
spec_dict['rotation'] = 45
spec_dict['groupby']  = [5,1]
spec_dict['lognorm']  = '10log10'
spec_dict['cmap']     = 'RdBu_r'
spec_dict['figname']  = 'secondLayerActivationsPSD'

for batch_ID in batch_IDs:
    fig = eegvislib.temporal_FM_PSD_layer2(input_window_normalized, 
                                           Mdl, layer,
                                           spec_dict, inout_labels,
                                           batch_ID, FM_ID)

    if save_img:
        fig.savefig(imgPath + spec_dict['figname'] + f'_batch{batch_ID}'+ f'_split{outFold}_{inFold}' + imgs_format,
                    transparent=False, bbox_inches='tight')

In [ ]:
# %matplotlib qt
batch_IDs = [100, 1000]
FM_ID = None #None #None (all FM) or a number 0 or list [0], [2,4]

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['FMnames'] = [i for i in spec_dict['bands'].keys()] #or None
spec_dict['backend'] = 'matplotlib' #matplotlib or qt
spec_dict['figname'] = 'temporalFMLayer2'

for batch_ID in batch_IDs:
    fig = eegvislib.temporal_FM_layer2(input_window_normalized,
                          Mdl, layer,
                          spec_dict,
                          batch_ID, FM_ID)
    if save_img:
        fig.savefig(imgPath + spec_dict['figname'] + f'_batch{batch_ID}'+ f'_split{outFold}_{inFold}' + imgs_format,
                    transparent=False, bbox_inches='tight')

### Batch Normalization

In [ ]:
layer = 'encoder.batch1'

ACTIVATION

In [ ]:
# %matplotlib qt
batch_IDs = [100, 1000]
FM_ID = None #None #None (all FM) or a number 0 or list [0], [2,4]

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['FMnames'] = [i for i in spec_dict['bands'].keys()] #or None
spec_dict['backend'] = 'matplotlib' #matplotlib or qt
spec_dict['figname'] = 'temporalFMLayer2'

for batch_ID in batch_IDs:
    fig = eegvislib.temporal_FM_layer2(input_window_normalized,
                          Mdl, layer,
                          spec_dict,
                          batch_ID, FM_ID)
    if save_img:
        fig.savefig(imgPath + spec_dict['figname'] + f'_batch{batch_ID}'+ f'_split{outFold}_{inFold}' + imgs_format,
                    transparent=False, bbox_inches='tight')

### Dense

In [ ]:
layer = 'Dense'

KERNEL

In [ ]:
spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']    = [5,2.5]
spec_dict['font']      = 14
spec_dict['title']     = f'Dense Layer Weights: Split {outFold}-{inFold}'
spec_dict['cbartick']  = 5
spec_dict['plot_type'] = 'both'
spec_dict['xticks']    = [i for i in spec_dict['bands'].keys()] #or None
spec_dict['groupby']   = 1
spec_dict['cmap']      = 'RdBu_r'
spec_dict['figname']   = 'denseLayerWeights'

fig, mask = eegvislib.denseweights_plot(Mdl_weights,layer,
                                        spec_dict)

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + f'{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

ACTIVATION

In [ ]:
layer = 'encoder'
convoluted_output = eegvislib.out_activation(Mdl, layer, input_window_normalized)
convoluted_output.shape
#batch x feature maps x scalp channels x timestep

In [ ]:
batch_IDs = [100, 1000]

spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']   = [15,1.5]
spec_dict['linew']    = 2
spec_dict['font']     = 20
spec_dict['rotation'] = 90
spec_dict['groupby']  = 1
spec_dict['cmap']     = 'RdBu_r'
spec_dict['figname']  = 'flattenLayerActivations'

for batch_ID in batch_IDs:
    fig = eegvislib.flatten_fm(input_window_normalized,
                               Mdl, layer, 
                               spec_dict, inout_labels,
                               batch_ID)
    if save_img:
        fig.savefig(imgPath + spec_dict['figname'] + f'_batch{batch_ID}' + f'_split{outFold}_{inFold}' + imgs_format,
                    transparent=False, bbox_inches='tight')

In [ ]:
spec_dict = eegvislib.get_spec_dict(spec_dict_keys, spec_dict_bands)
spec_dict['figdim']  = [5,5]
spec_dict['s']       = 10
spec_dict['loc']     = 'upper left'
spec_dict['font']    = 20
spec_dict['norm']    = 10
spec_dict['lognorm'] = '10log10'
spec_dict['pval']    = 0.05
spec_dict['method']  = 'holm'
spec_dict['figname'] = 'flattenCorrelationPSD'

fig = eegvislib.flattenCorrelationPSD(input_window_normalized, 
                                      Mdl, layer,
                                      spec_dict)

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

## Overlap

In [ ]:
layer = 'Dense'
input_window_normalized      = testloader.dataset[:][0]
convoluted_output_test       = eegvislib.out_activation(Mdl, layer, input_window_normalized)

validation_window_normalized = valloader.dataset[:][0]
convoluted_output_val        = eegvislib.out_activation(Mdl, layer, validation_window_normalized)

training_window_normalized   = trainloader.dataset[:][0]
convoluted_output_train      = eegvislib.out_activation(Mdl, layer, training_window_normalized)

In [ ]:
embeddingF   = torch.cat((convoluted_output_test, convoluted_output_val, convoluted_output_train), dim=0).numpy()
labels_test  = [classlabels[i] for i in testloader.dataset[: ][1].to(device = 'cpu').numpy()]
labels_val   = [classlabels[i] for i in valloader.dataset[ : ][1].to(device = 'cpu').numpy()]
labels_train = [classlabels[i] for i in trainloader.dataset[:][1].to(device = 'cpu').numpy()]
labels       = [labels_test, labels_val, labels_train]

### TRAIN-VALIDATION-TEST

2D VISUALIZATION

In [ ]:
spec_dict ={'title': '',
            'alpha_trainval': 0.5,
            's_trainval': 4,
            'alpha_test': 1,
            's_test': 10,
            'font': 16,
            'figdim': [10, 10],
            'marker': '.',
            'colors': ['tab:orange','blue', 'k'], #TEST, VAL, TRAIN
            'colors_lines': ['tab:red','violet', 'gray'], #TEST, VAL, TRAIN
            'loc': 'upper left',
            'linew': 2,
            'level': 0.3173,#0.046, #0.3173
            'gridsize': 50j,
            'sets': ['TRAIN','VAL','TEST'],
            'figname': 'overlap2Dsplit'}

# Main figure with subplots
dim = [[0, 1], [2, 1], [0, 2]]
fig, axes = plt.subplots(2, 2, figsize=(10, 10),
                         constrained_layout=True)
fig.suptitle(f'ACTIVATION 2D: Sets | Split {outFold}-{inFold}',
             fontsize=spec_dict['font']+2)

for i, ax in enumerate(axes.flat[:3]):
    spec_dict['dim'] = dim[i]
    spec_dict['xlabel'] = 'Output Activation ' + classlabels[dim[i][0]]
    spec_dict['ylabel'] = 'Output Activation ' + classlabels[dim[i][1]]
    eegvislib.embedding_split2D(embeddingF[:, dim[i]], labels, 
                                spec_dict, 
                                ax=ax)
    
fig.delaxes(axes[1, 1])

if save_img:
    fig.savefig(imgPath + spec_dict['figname']+ f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

3D VISUALIZATION

In [ ]:
spec_dict ={'title': f'ACTIVATION 3D: Sets | Split {outFold}-{inFold}',
            'alpha_trainval': 1,
            's_trainval': 1,
            'alpha_test': 1,
            's_test': 1,
            'font': 16,
            'figdim': [12, 12],
            'marker': '.',
            'loc': 'upper left',
            'cmap': ['Greys_r','Reds_r'],
            'view': {'elev':None,'azim':None,'roll':None},
            'level': 0.3173,
            'sigmalevels': np.arange(0,3.5,0.5),
            'sigmath': 1, #or None
            'gridsize': 50j,
            'sets': ['TRAIN','TEST'],
            'classlabels': classlabels,
            'figname': 'overlap3Dsplit'}

#%matplotlib qt

fig, ax, overlap = eegvislib.embedding_split3D(embeddingF, labels,
                                               spec_dict)
print(f'Overlap: {overlap:.1%} of {spec_dict['sets'][1]} set on {spec_dict['sets'][0]} set')

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

### LABELS

2D VISUALIZATION

In [ ]:
spec_dict = {'title': '',
             'alpha':0.5,
             's':4,
             'alpha_triangle': 0.7,
             'font':16,
             'figdim':[15,15],
             'marker':'.',
             'colors':['tab:green','tab:blue','tab:red'],
             'loc':'upper left',
             'linew': 3,
             'level': 0.3173,
             'sets': ['TEST','TRAIN'],
             'classlabels': classlabels,
             'figname': 'overlap2Dlabels'}

# Main figure with subplots
dim = [[0, 1], [2, 1], [0, 2]]
fig, axes = plt.subplots(2, 2, figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]),
                         constrained_layout=True)
fig.suptitle(f'ACTIVATION 2D: Labels | Split {outFold}-{inFold}',
             fontsize=spec_dict['font']+2)

for i, ax in enumerate(axes.flat[:3]):
    spec_dict['dim'] = dim[i]
    spec_dict['xlabel'] = 'Output Activation ' + classlabels[dim[i][0]]
    spec_dict['ylabel'] = 'Output Activation ' + classlabels[dim[i][1]]
    eegvislib.embedding_labels2D(embeddingF[:, dim[i]], labels, 
                                 spec_dict, 
                                 ax=ax)

fig.delaxes(axes[1, 1])

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

3D VISUALIZATION

In [ ]:
spec_dict ={'title': f'ACTIVATION 3D: Labels | Split {outFold}-{inFold}',
            'alpha': 0.4,
            's': 4,
            'alpha_triangle': 0.7,
            'font': 16,
            'figdim': [12, 12],
            'marker': '.',
            'colors':['tab:green','tab:blue','tab:red'],
            'loc': 'upper left',
            'linew': 2,
            'sets': ['TRAIN'],
            'classlabels': classlabels,
            'figname': 'overlap3Dlabels'}

#%matplotlib qt
fig, ax, baricenters, area = eegvislib.embedding_labels3D(embeddingF, labels,
                                                          spec_dict)

if save_img:
    fig.savefig(imgPath + spec_dict['figname'] + f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

### TEST SUBJECTS AND CLASSIFICATION MISTAKES

In [ ]:
nw_test  = np.concatenate(([testset.EEGcumlen[0]],  np.diff(testset.EEGcumlen)))

colors_test = []
for i in range(len(nw_test)):
    colors_test.extend([plt.cm.tab10.colors[i % 10]]*nw_test[i])

In [ ]:
spec_dict = {'title': '',
             'alpha_test':1,
             'alpha_trainval':0.5,
             's_test':20,
             's_trainval':4,
             'font':16,
             'figdim':[15,15],
             'marker':'.',
             'loc':'lower right',
             'loc_subj': 'lower left',
             'linew': 2,
             'level': 0.3173,
             'nsub': (len(partition_list[0][0]) + len(partition_list[0][1]) + len(partition_list[0][2])),
             'gridsize': 50j,
             'mode': 'subjects', #classification or subjects
             'figname': 'overlap2Dclassification'}

dim = [[0, 1], [2, 1], [0, 2]]
fig, axes = plt.subplots(2, 2, figsize=(spec_dict['figdim'][0], spec_dict['figdim'][1]),
                         constrained_layout=True)
fig.suptitle(f'ACTIVATION 2D: Classification Mistakes | Split {outFold}-{inFold}',
             fontsize=spec_dict['font']+2)

for i, ax in enumerate(axes.flat[:3]):
    spec_dict['xlabel'] = 'Output Activation ' + classlabels[dim[i][0]]
    spec_dict['ylabel'] = 'Output Activation ' + classlabels[dim[i][1]]
    eegvislib.embedding_class2D(embeddingF[:,dim[i]], inout_labels,
                                spec_dict, 
                                test_id, nw_test, colors_test, 
                                ax=ax)

fig.delaxes(axes[1, 1])

if save_img:
    fig.savefig(imgPath + spec_dict['figname']+ f'_split{outFold}_{inFold}' + imgs_format,
                transparent=False, bbox_inches='tight')

In [ ]:
importlib.reload(eegvislib)